<div style="color:white; background-color:#5642C5; padding: 10px; border-radius: 15px; font-size: 150%; font-family: Verdana; text-align:center; -webkit-text-stroke-width: 1px; -webkit-text-stroke-color: black; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7);"> 🔥🔥IMPORTING LIBRARIES🔥🔥 </div>

In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/raw-data-practice/rawdata.xlsx


<div style="color:white; background-color:#5642C5; padding: 10px; border-radius: 15px; font-size: 150%; font-family: Verdana; text-align:center; -webkit-text-stroke-width: 1px; -webkit-text-stroke-color: black; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7);"> 🔥🔥TRAIN SAMPLE DATA🔥🔥 </div>

In [2]:
raw_data = pd.read_excel("/kaggle/input/raw-data-practice/rawdata.xlsx")

In [3]:
print("Data Loaded")
raw_data.sample(5).style.set_properties(**{'background-color': '#f9f9f9', 'color': '#4CAF50', 'font-weight': 'bold'})

Data Loaded


,date,time,sensor,location,number,activity,position,location.1
9,2024-01-16 00:00:00,11:26:20,0.000000,A10,1,picked,inside,A10
123,2024-01-18 00:00:00,14:28:20,0.000000,A4,1,picked,inside,A4
53,2024-01-16 00:00:00,00:57:20,0.000000,A26,1,picked,inside,A26
136,2024-01-18 00:00:00,16:42:00,1.000000,A17,1,placed,inside,A17
38,2024-01-16 00:00:00,00:35:20,0.000000,A11,1,picked,inside,A11


In [4]:
raw_data.columns

Index(['date', 'time', 'sensor', 'location', 'number', 'activity', 'position',
       'location.1'],
      dtype='object')

In [5]:
# Convert 'date' column to datetime format
raw_data['date'] = pd.to_datetime(raw_data['date'])

In [6]:
# Check the data types
print("\nData Types:")
print(raw_data.dtypes)


Data Types:
date          datetime64[ns]
time                  object
sensor               float64
location              object
number                 int64
activity              object
position              object
location.1            object
dtype: object


In [7]:
# Create a column for the assumed duration
raw_data['duration'] = 1

<div style="color:white; background-color:#5642C5; padding: 10px; border-radius: 15px; font-size: 150%; font-family: Verdana; text-align:center; -webkit-text-stroke-width: 1px; -webkit-text-stroke-color: black; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7);"> 🔥🔥Part-1🔥🔥 </div>

In [8]:
# Pivot table to sum durations for 'inside' and 'outside'
duration_data = raw_data.pivot_table(index='date', columns='position', values='duration', aggfunc='sum', fill_value=0)

In [9]:
# Rename columns
duration_data = duration_data.rename(columns={'inside': 'inside_duration', 'outside': 'outside_duration'})

In [10]:
# Check intermediate duration_data
print("\nDuration Data:")
print(duration_data.head())


Duration Data:
position    Inside  inside_duration  outside_duration
date                                                 
2024-01-16       2               78                 0
2024-01-17       0               19                 0
2024-01-18      13               59                 4


<div style="color:white; background-color:#5642C5; padding: 10px; border-radius: 15px; font-size: 150%; font-family: Verdana; text-align:center; -webkit-text-stroke-width: 1px; -webkit-text-stroke-color: black; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7);"> 🔥🔥Part-2🔥🔥 </div>

In [11]:
# Step 3: Count the number of 'picking' and 'placing' activities for each date
activity_count = raw_data.groupby(['date', 'activity']).size().unstack(fill_value=0)

In [12]:
# Rename columns
activity_count = activity_count.rename(columns={'picking': 'pick_activities', 'placing': 'place_activities'})

In [13]:
# Check intermediate activity_count
print("\nActivity Count Data:")
print(activity_count.head())


Activity Count Data:
activity    picked  placed
date                      
2024-01-16      40      40
2024-01-17      10       9
2024-01-18      37      39


<div style="color:white; background-color:#5642C5; padding: 10px; border-radius: 15px; font-size: 150%; font-family: Verdana; text-align:center; -webkit-text-stroke-width: 1px; -webkit-text-stroke-color: black; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7);"> 🔥🔥Data Merge🔥🔥 </div>

In [14]:
# Step 4: Merge duration and activity count data
output_data = pd.concat([duration_data, activity_count], axis=1).fillna(0)

In [15]:
# Ensure all required columns are present
for column in ['inside_duration', 'outside_duration', 'pick_activities', 'place_activities']:
    if column not in output_data:
        output_data[column] = 0

In [16]:
# Step 5: Reset index to have 'date' as a column and select only required columns
output_data = output_data.reset_index()
output_data = output_data[['date', 'pick_activities', 'place_activities', 'inside_duration', 'outside_duration']]

In [17]:
# Check final output data
print("\nFinal Output Data:")
print(output_data.head())


Final Output Data:
        date  pick_activities  place_activities  inside_duration  \
0 2024-01-16                0                 0               78   
1 2024-01-17                0                 0               19   
2 2024-01-18                0                 0               59   

   outside_duration  
0                 0  
1                 0  
2                 4  


<div style="color:white; background-color:#5642C5; padding: 10px; border-radius: 15px; font-size: 150%; font-family: Verdana; text-align:center; -webkit-text-stroke-width: 1px; -webkit-text-stroke-color: black; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7);"> 🔥🔥File Saving🔥🔥 </div>

In [18]:
# Write the processed data to a CSV file
output_data.to_csv("output.csv", index=False)